<img src="./assets/images/iris.jpg" alt="iris" title="iris" style="float: right; margin-left: 1rem;" />

# scikit-learn Iris Model using jsonData

We will train a model for the Iris classification task. In this simple dataset we try to classifier a type if Iris based on its petal and sepal length. 

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube

## Dependencies

 * [S2I](https://github.com/openshift/source-to-image)
 * [sklearn](http://scikit-learn.org/stable/) to train the model
 * [grpcio-tools](https://grpc.io/docs/quickstart/python.html) to allow testing using gRPC
 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)

In [9]:
!pygmentize ./IrisClassifier.py

import joblib
import sys


def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)


class IrisClassifier(object):
    def __init__(self):
        self.model = joblib.load('IrisClassifier.sav')
        # self.class_names = ["iris-setosa","iris-vericolor","iris-virginica"]

    def predict(self, X, features_names):
        eprint('--------------------')
        eprint('Input dict')
        eprint(X)
        eprint('--------------------')

        return self.model.predict_proba(X)


In [10]:
!cat .s2i/environment

MODEL_NAME=IrisClassifier
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0


In [12]:
!pygmentize ./contract.json

{
  "features": [
    {
      "name": "sepal_length",
      "dtype": "FLOAT",
      "ftype": "continuous",
      "range": [
        4,
        8
      ]
    },
    {
      "name": "sepal_width",
      "dtype": "FLOAT",
      "ftype": "continuous",
      "range": [
        2,
        5
      ]
    },
    {
      "name": "petal_length",
      "dtype": "FLOAT",
      "ftype": "continuous",
      "range": [
        1,
        10
      ]
    },
    {
      "name": "petal_width",
      "dtype": "FLOAT",
      "ftype": "continuous",
      "range": [
        0,
        3
      ]
    }
  ],
  "targets": [
    {
      "name": "class",
      "dtype": "FLOAT",
      "ftype": "continuous",
      "range": [
        0,
        1
      ],
      "repeat": 3
    }
  ]
}


In [11]:
!pygmentize deployment.json

{
  "apiVersion": "machinelearning.seldon.io/v1alpha2",
  "kind": "SeldonDeployment",
  "metadata": {
    "labels": {
      "app": "seldon"
    },
    "name": "sklearn-iris-classifier"
  },
  "spec": {
    "annotations": {
      "project_name": "Iris classification",
      "deployment_version": "0.1"
    },
    "name": "sklearn-iris-deployment",
    "oauth_key": "oauth-key",
    "oauth_secret": "oauth-secret",
    "predictors": [
      {
        "componentSpecs": [
          {
            "spec": {
              "containers": [
                {
                  "image": "brunowego/sklearn-iris-jsondata:0.1",
                  "imagePullPolicy": "IfNotPresent",
                  "name": "sklearn-iris-classifier",
                  "resources": {
                    "requests": {
                      "memory": "1Mi"
                    }
                  }
                }
              ],
              "terminationGracePeriodSeconds": 20
            }
          }
        ],
       